In [41]:
import numpy as np
import os
import glob
import nibabel as nib
import bids
import matplotlib.pyplot as plt

# Denoise correction
from dipy.denoise.patch2self import patch2self

# motion correction
from dipy.align import motion_correction 
from dipy.core.gradients import gradient_table
from dipy.io.gradients import read_bvals_bvecs
from dipy.io.image import load_nifti, save_nifti


In [20]:
path_dwi_raw = '/opt/dora/Dora/BIDS_ELA/BIDS_ELA/'
path_dwi_preproc = '/opt/dora/Dora/dwi_scripts/1_preproc'

In [21]:
dwi_raw_bids = bids.BIDSLayout(path_dwi_raw)

# Get all DWI images from the BIDS dataset
dwi_images = dwi_raw_bids.get(suffix='dwi', extension='.nii.gz')
dwi_bvals = dwi_raw_bids.get(suffix='dwi', extension='.bval')
dwi_bvecs = dwi_raw_bids.get(suffix='dwi', extension='.bvec')

In [22]:
# Print the number of DWI images found
print(f"Found {len(dwi_images)} DWI images")
print(f"Found {len(dwi_bvals)} DWI b-values")
print(f"Found {len(dwi_bvecs)} DWI b-vectors")

Found 95 DWI images
Found 95 DWI b-values
Found 95 DWI b-vectors


In [42]:
# preprocessing of image
for dwi_img, bval, bvec in list(zip(dwi_images, dwi_bvals, dwi_bvecs))[:1]:
    # Load the DWI image
    data, affine = load_nifti(dwi_img.path)
    bvals, bvecs = read_bvals_bvecs(bval.path, bvec.path)
    gtab = gradient_table(bvals, bvecs)
    # denoise correction 
    denoise_data = patch2self(data, bvals=bvals, model='ols', shift_intensity=True, clip_negative_vals=False, b0_threshold=50)
